In [1]:
import PySimpleGUI as sg
import configparser
import os
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from prophet import Prophet
from io import BytesIO
import threading
import time
from arch import arch_model
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from yahoo_fin import stock_info as si
import requests
import pandas as pd
from io import StringIO
from yahooquery import Ticker
from googletrans import Translator
import csv
import tkinter as tk
from tkinter import messagebox
import matplotlib.colors as mcolors

# Функция для чтения CSV файла
def load_data_from_csv(file_path):
    companies_data = {}
    tickers = []
    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file, delimiter=';')  # Указываем разделитель
        reader.fieldnames = [name.lstrip('\ufeff') for name in reader.fieldnames]  # Удаляем BOM
        print("Заголовки столбцов:", reader.fieldnames)  # Проверяем заголовки
        for row in reader:
            try:
                tickers.append(row['ticker'])  # Названия столбцов из заголовков
                companies_data[row['ticker']] = {
                    'company': row['company'],
                    'sector': row['sector'],
                    'industry': row['industry'],
                    'description': row['description']
                }
            except KeyError as e:
                print(f"Ошибка: отсутствует столбец {e} в CSV.")
                raise
    return tickers, companies_data


# Загрузка данных из CSV
tickers, companies_data = load_data_from_csv('companies.csv')


# Пример данных о компаниях
def get_all_stocks():
    return ['AAPL', 'MSFT', 'GOOGL', 'JPM', 'BAC', 'XOM']  # Список тикеров


# Изначальные данные об акциях по отраслям
def get_sector_stocks():
    return {
        'Технологии': ['AAPL', 'MSFT', 'GOOGL'],
        'Финансы': ['JPM', 'BAC', 'C'],
        'Энергетика': ['XOM', 'CVX', 'BP']
    }


# Функция для сохранения выбранных акций в файл settings.ini
def save_selected_stocks(selected_stocks, file_name='settings.ini'):
    try:
        config = configparser.ConfigParser()
        for sector, stocks in selected_stocks.items():
            config[sector] = {'акции': ','.join(stocks)}
        with open(file_name, 'w', encoding='utf-8') as configfile:
            config.write(configfile)
        sg.popup('Выбранные акции успешно сохранены в settings.ini')
    except Exception as e:
        sg.popup(f"Ошибка при сохранении файла: {e}")

# Функция для загрузки выбранных акций из файла settings.ini
def load_selected_stocks(file_name='settings.ini'):
    config = configparser.ConfigParser()
    selected_stocks = {sector: [] for sector in get_sector_stocks().keys()}
    if os.path.exists(file_name):
        try:
            config.read(file_name, encoding='utf-8')
            for sector in config.sections():
                if 'акции' in config[sector]:
                    stocks = config[sector]['акции'].strip()
                    if stocks:
                        selected_stocks[sector] = stocks.split(',')
        except Exception as e:
            sg.popup(f"Ошибка при загрузке файла настроек: {e}")
    return selected_stocks

# Инициализация данных
original_sector_stocks = get_sector_stocks()  # Исходные данные (для возврата акций)
current_sector_stocks = get_sector_stocks()  # Текущее состояние доступных акций

# Загрузка сохраненных данных из файла (если он существует)
selected_stocks = load_selected_stocks()  # Загружаем сохраненные выбранные акции из файла

undo_stack = []

# Убираем уже выбранные акции из доступных
def update_current_available_stocks():
    global current_sector_stocks
    for sector, selected in selected_stocks.items():
        current_sector_stocks[sector] = [stock for stock in current_sector_stocks[sector] if stock not in selected]

# Обновляем доступные акции при загрузке
update_current_available_stocks()

# Функции для отображения доступных и выбранных акций
def update_available_display(selected_sector):
    # Обновление списка доступных акций в зависимости от выбранной отрасли
    display_list = []
    if selected_sector == 'Все отрасли':
        for sector, stocks in current_sector_stocks.items():
            if stocks:
                display_list.append(f"{sector}:")
                display_list.extend(stocks)
                display_list.append("")  # Пустая строка для визуального разделения
    else:
        stocks = current_sector_stocks.get(selected_sector, [])
        if stocks:
            display_list.append(f"{selected_sector}:")
            display_list.extend(stocks)

    window['-AVAILABLE-'].update(display_list)

def update_selected_display():
    # Обновление списка выбранных акций с секциями
    display_list = []
    for sector, stocks in selected_stocks.items():
        if stocks:
            display_list.append(f"{sector}:")
            display_list.extend(stocks)
            display_list.append("")  # Пустая строка для визуального разделения

    window['-SELECTED-'].update(display_list)

# --- Функции для оптимизации портфеля ---
def fetch_data(tickers, start_date, end_date):
    data = {}
    for ticker in tickers:
        try:
            data[ticker] = yf.download(ticker, start=start_date, end=end_date)['Adj Close']
        except Exception as e:
            print(f"Не удалось загрузить данные для {ticker}: {e}")
            data[ticker] = pd.Series()
    return data

def prepare_data_for_prophet(data, ticker):
    df = data[ticker].reset_index()
    df.columns = ['ds', 'y']
    df = df.dropna()
    if len(df) < 2:
        raise ValueError(f"Недостаточно данных для тикера {ticker}. Нужно хотя бы 2 строки с данными.")
    return df

def forecast_prices(data, ticker, days_ahead=30):
    try:
        df = prepare_data_for_prophet(data, ticker)
        model = Prophet()
        model.fit(df)
        future = model.make_future_dataframe(periods=days_ahead)
        forecast = model.predict(future)
        return forecast[['ds', 'yhat']].tail(days_ahead)
    except ValueError as e:
        sg.popup(f"Ошибка при прогнозировании для {ticker}: {e}")
        return pd.DataFrame(columns=['ds', 'yhat'])

def portfolio_performance(weights, expected_returns, cov_matrix):
    # Функция для оценки производительности портфеля
    portfolio_return = np.dot(weights, expected_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    portfolio_sharpe = portfolio_return / portfolio_volatility  # Используем без учета безрисковой ставки
    return portfolio_return, portfolio_volatility, portfolio_sharpe

# Обновление тикеров на второй вкладке, где осуществляется расчет оптимальных весов
def update_tickers_input():
    # Обновляем форматированный список по отраслям
    formatted_tickers = format_tickers_display()
    window['tickers_display'].update(formatted_tickers)  # Обновляем только этот элемент

# Функция для создания круговой диаграммы



# Обновляем поле tickers на второй вкладке перед запуском окна
def initialize_tickers_input():
    # Собираем тикеры из сохранённых акций
    selected_tickers = []
    for sector, stocks in selected_stocks.items():
        selected_tickers.extend(stocks)
    return ', '.join(selected_tickers)

# Функция для обновления тикеров на вкладке "Дополнительно"
def update_selected_tickers_display():
    # Собираем тикеры по отраслям
    selected_tickers_display = ""
    for sector, stocks in selected_stocks.items():
        if stocks:  # Проверяем, есть ли акции в отрасли
            selected_tickers_display += f"{sector}:\n"
            selected_tickers_display += '\n'.join(stocks) + "\n\n"
    
    # Обновляем поле на вкладке "Дополнительно"
    window['-SELECTED-TICKERS-'].update(selected_tickers_display)

# Форматируем тикеры для отображения с разделением на отрасли
def format_tickers_display():
    display_str = ""
    for sector, stocks in selected_stocks.items():
        if stocks:  # Только если в секторе есть акции
            display_str += f"{sector}:\n" + ', '.join(stocks) + "\n\n"
    return display_str.strip()

def predict_returns_with_ml(tickers, start_date, end_date):
    # Загружаем данные о ценах
    data = fetch_data(tickers, start_date, end_date)
    predictions = {}

    for ticker in tickers:
        if ticker in data and not data[ticker].empty:
            # Расчет ежедневных доходностей
            daily_returns = data[ticker].pct_change().dropna()

            # Используем предыдущие 5 дней для предсказания следующего дня
            X = []
            y = []

            # Формируем данные для машинного обучения
            for i in range(5, len(daily_returns)):
                X.append(daily_returns[i-5:i].values)  # Сдвигаем 5 дней
                y.append(daily_returns.iloc[i])  # Текущая доходность

            X = np.array(X)
            y = np.array(y)

            # Разделяем данные на обучающую и тестовую выборки
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

            # Обучаем модель
            model = LinearRegression()
            model.fit(X_train, y_train)

            # Делаем предсказание
            predicted_returns = model.predict(X_test)

            # Добавляем прогнозируемую доходность в словарь
            predictions[ticker] = predicted_returns[-1]  # Прогнозируем последнюю доходность

    return predictions

# Функция для расчета доходности и риска с использованием машинного обучения
def calculate_returns_and_risks_ml(tickers, start_date, end_date):
    # Получаем предсказания доходности с помощью ML
    predicted_returns = predict_returns_with_ml(tickers, start_date, end_date)
    
    # Словарь для хранения результатов
    results = {}
    
    for ticker in tickers:
        if ticker in predicted_returns:
            # Прогнозируемая доходность
            predicted_return = predicted_returns[ticker]

            # Ожидаемая волатильность на основе стандартного отклонения
            daily_returns = fetch_data([ticker], start_date, end_date)[ticker].pct_change().dropna()
            annual_volatility = daily_returns.std() * np.sqrt(252)  # Годовая волатильность

            results[ticker] = {
                'return': round(predicted_return * 100, 2),  # Переводим в проценты
                'risk': round(annual_volatility * 100, 2)  # Переводим в проценты
            }
        else:
            results[ticker] = {'return': None, 'risk': None}
    
    return results


# Получаем все тикеры из функции get_sector_stocks()
sector_stocks = [stock for stocks in get_sector_stocks().values() for stock in stocks]


layout_tab1 = [
    [
        sg.Column([  # Левая часть
            [
                sg.Text('Выберите отрасль:', font=('Segoe UI', 16), expand_x=True),
                sg.Combo(['Все отрасли'] + list(original_sector_stocks.keys()), key='-SECTOR-', default_value='Все отрасли', readonly=True, font=('Segoe UI', 12), size=(44, 5), expand_x=True)
            ],
            [
                sg.Button('Применить фильтр', key='-FILTER-', font=('Segoe UI', 12), size=(31, 2), expand_x=True),
                sg.Button('Выбрать все акции всех отраслей', key='-SELECT-ALL-', font=('Segoe UI', 12), size=(31, 2), expand_x=True)
            ],
            [
                sg.Button('Выбрать все акции всех отраслей', key='-SELECT-ALL-', font=('Segoe UI', 12), size=(31, 2), expand_x=True),
                sg.Button('Очистить список выбранных', key='-CLEAR-SELECTED-', font=('Segoe UI', 12), size=(31, 2), expand_x=True)
            ],
            [sg.Text('Доступные акции:', font=('Segoe UI', 14), expand_x=True), sg.Text('Выбранные акции:', font=('Segoe UI', 14), expand_x=True)],
            [
                sg.Listbox(values=[], font=('Segoe UI', 12), key='-AVAILABLE-', enable_events=True, size=(31, 15), expand_x=True),  # Средний размер для списка
                sg.Listbox(values=[], font=('Segoe UI', 12), key='-SELECTED-', enable_events=True, size=(31, 15), expand_x=True)  # Средний размер для списка
            ],
            [sg.Button('Отменить последний выбор', key='-UNDO-', font=('Segoe UI', 12), size=(31, 2), expand_x=True)],
            [sg.Button('Сохранить выбор', key='-SAVE-', font=('Segoe UI', 12), size=(31, 2), expand_x=True)]
        ], size=(620, 720), expand_x=True, element_justification='left', pad=(0, 0, 0, 0)),  # Левая часть
        sg.VerticalSeparator(),  # Разделитель (1px)
        sg.Column([  # Правая часть
            [sg.Text('Информация о компаниях', font=('Segoe UI', 16), justification='center', size=(50, 1), expand_x=True)],  # Заголовок по центру
            [
                sg.Combo(tickers, key='-COMPANY-', font=('Segoe UI', 12), readonly=True, size=(44, 5), expand_x=True)
            ],  # Выпадающий список с тикерами
            [
                sg.InputText('', key='-STOCK-', font=('Segoe UI', 12), size=(31, 2), expand_x=True)
            ],  # Окно ввода для названия акции
            [
                sg.Button('Поиск', key='-SEARCH-', font=('Segoe UI', 12), size=(31, 2), expand_x=True)
            ],  # Кнопка поиска
            [
                sg.Button('Получить информацию о компании', key='-GET-INFO-', font=('Segoe UI', 12), size=(31, 2), expand_x=True)
            ],  # Новая кнопка для получения информации
            [
                sg.Multiline('', key='-COMPANY-INFO-', font=('Segoe UI', 12), size=(50, 20), expand_x=True, autoscroll=True, no_scrollbar=False, disabled=True)
            ]  # Используем sg.Multiline для информации о компании
        ], size=(620, 720), expand_x=True, element_justification='right', pad=(0, 0, 0, 0)),  # Правая часть
    ]
]


layout_tab2 = [
    [
        sg.Column([  # Левая часть
            [sg.Text('Выбранные акции (разделены по отраслям):', font=('Segoe UI', 14), expand_x=True)],
            [sg.Multiline(format_tickers_display(), size=(67, 16), key='tickers_display', font=('Segoe UI', 12), disabled=True, expand_x=True, no_scrollbar=True)],  # Поле только для чтения
            [sg.Text('Начальная дата (YYYY-MM-DD):', font=('Segoe UI', 14), expand_x=True)],
            [sg.InputText('2018-01-01', key='start_date', font=('Segoe UI', 12), size=(67, 1), expand_x=True)],
            [sg.CalendarButton('Выбрать дату', target='start_date', format='%Y-%m-%d', font=('Segoe UI', 12), size=(67, 1))],
            [sg.Text('Конечная дата (YYYY-MM-DD):', font=('Segoe UI', 14), expand_x=True)],
            [sg.InputText('2024-12-05', key='end_date', font=('Segoe UI', 12), size=(67, 1), expand_x=True)],
            [sg.CalendarButton('Выбрать дату', target='end_date', format='%Y-%m-%d', font=('Segoe UI', 12), size=(67, 1))],
            [sg.Button('Рассчитать оптимальные веса', font=('Segoe UI', 12), size=(31, 2), expand_x=True)],
        ], size=(620, 720), expand_x=True, element_justification='left', pad=(0, 0, 0, 0)),  # Левая колонка
        sg.VerticalSeparator(),  # Разделитель
        sg.Column([  # Правая часть
            [sg.Text('Процесс выполнения:', font=('Segoe UI', 14), expand_x=True)],
            [sg.Multiline(size=(65, 13), key='-LOG-', font=('Segoe UI', 12), autoscroll=True, disabled=True, expand_x=True)],  # Лог выполнения
            [sg.Text('Соотношение доходов и рисков:', font=('Segoe UI', 14), expand_x=True)],
            [sg.Multiline(size=(65, 14), key='-RISK-RETURN-', font=('Segoe UI', 12), disabled=True, expand_x=True)],  # Окно для доходности и риска
        ], size=(620, 720), expand_x=True, element_justification='left', pad=(0, 0, 0, 0)),  # Правая колонка
    ]
]

layout_tab3 = [
    [
        sg.Column([  # Левая часть
            [sg.Text('Оптимизация:', font=('Segoe UI', 15), justification='center', size=(50, 1), expand_x=True)],
            [sg.Text('Проанализированные акции:', font=('Segoe UI', 14), size=(55, 1), expand_x=True)],
            [sg.Multiline('', key='-ANALYZED-STOCKS-', font=('Segoe UI', 12), size=(50, 10), expand_x=True, autoscroll=True, disabled=True, no_scrollbar=False)],
            [sg.Button('Продолжить оптимизацию', key='-CONTINUE-OPTIMIZATION-', font=('Segoe UI', 12), size=(31, 2), expand_x=True, disabled=True)],
            [sg.Text('Результаты оптимизации:', font=('Segoe UI', 14), size=(50, 1), expand_x=True)],
            [sg.Multiline('', key='-RESULTS-', font=('Segoe UI', 12), size=(50, 10), expand_x=True, autoscroll=True, disabled=True, no_scrollbar=False)],
            [sg.Button('Сохранить результаты в папку', key='-SAVE-RESULTS-', font=('Segoe UI', 12), size=(31, 2), expand_x=True)]  # Новая кнопка
        ], size=(620, 720), expand_x=True, element_justification='center', pad=(0, 0, 0, 0)),
        
        sg.VerticalSeparator(),  # Разделитель (1px)
        
        sg.Column([  # Правая часть
            [sg.Text('Диаграммы и Графики', font=('Arial', 16, 'bold'), justification='center', size=(50, 1))],
            [sg.Text('Диаграмма оптимального портфеля:', justification='left',font=('Arial', 12))],
            # Используем фиксированный размер для диаграммы, чтобы не сдвигать элементы
            [sg.Image(key='-PIE-', pad=(0, 0))],  # Фиксированный размер
        ], size=(620, 720), expand_x=False, element_justification='left', pad=(0, 0, 0, 0)),  # Правая часть
    ]
]

# Создание окна с вкладками
tab1 = sg.Tab('Фильтрация акций', layout_tab1)
tab2 = sg.Tab('Оптимизация портфеля', layout_tab2)
tab3 = sg.Tab('Аналитика акций', layout_tab3)

layout = [
    [sg.TabGroup([[tab1, tab2, tab3]])]
]

# Создание окна с layout
window = sg.Window("Финансовый помощник", layout, size=(1280, 720), finalize=True)


# Функция для имитации процесса
def perform_calculations(window):
    for i in range(10):
        time.sleep(1)  # Имитация времени работы
        log_message = f"[{i+1}/10] Расчет {i+1} завершен"
        window['-LOG-'].print(log_message)  # Обновление окна лога
        window.write_event_value('-UPDATE_LOG-', log_message)  # Это может быть полезно для обновления других частей интерфейса
    # Финальный результат
    window['-LOG-'].print("Все расчеты завершены!")


# Инициализация отображения данных
update_available_display(window['-SECTOR-'].get())  # Обновляем доступные акции
update_selected_display()  # Обновляем выбранные акции

# Функция для подготовки данных для обучения
def prepare_data_for_training(data):
    X = []
    y = []
    for ticker in data:
        ticker_data = data[ticker].values  # Преобразуем в массив
        print(f"Initial ticker data for {ticker}: {ticker_data.shape}")  # Логируем форму данных
        
        if ticker_data.ndim == 1:  # Если данные одномерные, преобразуем их в двумерный массив
            ticker_data = ticker_data.reshape(-1, 1)  # Преобразуем в (N, 1)
        
        for i in range(60, len(ticker_data)):  # Делаем окно на 60 дней
            X.append(ticker_data[i-60:i, 0])  # Последние 60 дней
            y.append(ticker_data[i, 0])  # Цена на следующий день

    X = np.array(X)
    y = np.array(y)
    
    print(f"X shape after preparation: {X.shape}")  # Логируем форму X
    print(f"y shape after preparation: {y.shape}")  # Логируем форму y
    
    # Преобразуем X в двумерный массив (samples, features)
    X = np.reshape(X, (X.shape[0], -1))  # Преобразуем в двумерный массив (samples, features)
    print(f"X shape after reshaping: {X.shape}")  # Логируем форму X после reshape
    
    return X, y



# Обучение модели
def train_model(X, y):
    try:
        print(f"Training data X shape: {X.shape}")  # Логируем форму X
        print(f"Training data y shape: {y.shape}")  # Логируем форму y

        # Проверка: если X не двумерный, вывести ошибку
        if X.ndim != 2:
            raise ValueError(f"X must be a 2D array. Got {X.ndim}D instead.")
        
        model = LinearRegression()
        model.fit(X, y)  # Обучаем модель
        
        return model
    except Exception as e:
        print(f"Error in train_model: {e}")
        raise  # Повторно выбрасываем исключение для дальнейшей отладки




def portfolio_risk(weights, returns, cov_matrix):
    # Вычисление риска (волатильности) портфеля
    portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))  # Дисперсия портфеля
    portfolio_volatility = np.sqrt(portfolio_variance)  # Стандартное отклонение (волатильность)
    return portfolio_volatility


def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))  # Выходное значение - прогнозируемая цена
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def train_lstm_model(data, window_size=60):
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i - window_size:i, 0])
        y.append(data[i, 0])
    X = np.array(X)
    y = np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))  # Преобразование для LSTM
    model = create_lstm_model((X.shape[1], 1))
    model.fit(X, y, epochs=10, batch_size=32)
    return model

def predict_future_prices(model, last_60_days):
    try:
        last_60_days_scaled = np.array(last_60_days).reshape(-1, 1)  # Преобразуем в 2D для модели линейной регрессии
        print(f"Last 60 days shape: {last_60_days_scaled.shape}")  # Логируем форму
        
        future_predictions = []
        
        for _ in range(30):
            # Прогнозируем на 1 день вперед
            prediction = model.predict(last_60_days_scaled[-60:].reshape(1, -1))  # Преобразуем в (1, 60)
            print(f"Prediction shape: {prediction.shape}")  # Логируем форму предсказания
            
            future_predictions.append(prediction[0])  # Добавляем предсказание
            
            # Преобразуем предсказание в двумерный массив (1, 1) и добавляем его к последним 60 дням
            prediction = prediction.reshape(1, 1)  # Преобразуем в (1, 1)
            last_60_days_scaled = np.append(last_60_days_scaled, prediction, axis=0)  # Обновляем данные

        return future_predictions
    except Exception as e:
        print(f"Error in predict_future_prices: {e}")
        raise  # Повторно выбрасываем исключение для дальнейшей отладки





def calculate_volatility(data):
    returns = data.pct_change().dropna()
    model = arch_model(returns, vol='Garch', p=1, q=1)
    model_fit = model.fit(disp="off")
    return model_fit.conditional_volatility[-1]  # Волатильность на последний день

def optimize_portfolio(tickers, returns, risks, threshold=0.02):
    # Подготовка данных для оптимизации
    expected_returns = np.array([returns[ticker] for ticker in tickers])
    expected_risks = np.array([risks[ticker] for ticker in tickers])

    if expected_returns.ndim != 1 or expected_risks.ndim != 1:
        raise ValueError("Returns and risks must be 1-dimensional arrays.")

    # Считаем ковариационную матрицу
    cov_matrix = np.diag(expected_risks ** 2)  # Диагональная матрица для ковариации

    num_assets = len(tickers)
    initial_weights = np.array([1/num_assets] * num_assets)
    bounds = [(0, 1) for _ in range(num_assets)]
    constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}

    # Минимизация функции отрицательной метрики Шарпа
    result = minimize(
        lambda weights: -portfolio_performance(weights, expected_returns, cov_matrix)[2],  # минимизация отрицательной Шарпа
        initial_weights, method='SLSQP', bounds=bounds, constraints=constraints
    )

    # Вес для каждого тикера
    weights = result.x
    weights_dict = {ticker: weight for ticker, weight in zip(tickers, weights)}

    # Получаем результаты оптимизации
    opt_return, opt_volatility, opt_sharpe = portfolio_performance(weights, expected_returns, cov_matrix)

    # Генерация круговой диаграммы с фильтрацией по весу
    fig, ax = plt.subplots(figsize=(7.5, 5.9))  # Сохраняем размер диаграммы 7.5x6.0

    # Создаем список для всех акций с весом больше порога и для группы "Прочие"
    sizes = []
    labels = []
    colors = []
    
    # Суммируем веса акций с маленьким весом в одну группу
    other_weight = 0
    other_label = "Прочие"
    other_color = '#CCCCCC'  # Серый цвет для группы "Прочие"

    for i, weight in enumerate(weights):
        if weight >= threshold:
            sizes.append(weight)
            labels.append(tickers[i])
            colors.append(plt.cm.tab20(i % 20))  # Используем цвет из палитры 'tab20'
        else:
            other_weight += weight

    # Если есть акции с маленьким весом, добавляем их в "Прочие"
    if other_weight > 0:
        sizes.append(other_weight)
        labels.append(other_label)
        colors.append(other_color)

    # Создаем диаграмму с процентами, размещенными на внешней части
    wedges, texts, autotexts = ax.pie(
        sizes, startangle=90, wedgeprops={'width': 0.3}, colors=colors, 
        autopct='%1.1f%%', textprops={'fontsize': 7}, pctdistance=1.15, labeldistance=1.2  # Перемещаем проценты наружу
    )
    ax.axis('equal')  # Равные оси для круга

    # Перемещаем легенду (палитру) немного ближе к диаграмме
    ax.legend(wedges, labels, title="Компании", loc="upper center", bbox_to_anchor=(0.5, -0.1), ncol=3, fontsize=8)

    # Возвращаем изображение в формате PNG
    buf = BytesIO()
    plt.savefig(buf, format='png', dpi=100, bbox_inches='tight')  # Используем bbox_inches='tight' для плотного размещения
    buf.seek(0)
    img_data = buf.read()
    buf.close()

    # Возвращаем 5 значений
    return weights_dict, opt_return, opt_volatility, opt_sharpe, img_data


def perform_optimization_in_thread(window, tickers, start_date, end_date):
    try:
        # Загрузка данных с использованием введенных дат
        data = fetch_data(tickers, start_date, end_date)

        # Подготовка данных для обучения
        X, y = prepare_data_for_training(data)

        # Обучение модели
        model = train_model(X, y)

        # Прогнозирование на 30 дней вперед для каждого тикера
        future_predictions = {}
        for ticker in tickers:
            if ticker in data:
                # Берем последние 60 дней для прогноза
                last_60_days = data[ticker].values[-60:]

                # Прогнозируем на 30 дней вперед
                predictions = predict_future_prices(model, last_60_days)

                future_predictions[ticker] = predictions

        # Рассчитываем доходность и риск на основе прогнозируемых цен
        returns = {}
        risks = {}
        for ticker, predictions in future_predictions.items():
            # Доходность на основе прогнозируемых цен (в конце и в начале периода)
            return_ = (predictions[-1] - predictions[0]) / predictions[0]  # Прогнозируемая доходность

            # Риск (стандартное отклонение) на основе прогнозируемых цен
            risk_ = np.std(np.diff(predictions))  # Стандартное отклонение изменений (доходности)

            returns[ticker] = return_
            risks[ticker] = risk_

        # Оптимизация портфеля (например, с использованием минимизации риска)
        weights, opt_return, opt_volatility, opt_sharpe, pie_chart_data = optimize_portfolio(tickers, returns, risks)

        # Форматирование результатов для отображения
        weight_str = ", ".join([f"{ticker}: {weight:.2f}" for ticker, weight in weights.items()])
        ret = sum(returns[ticker] * weights[ticker] for ticker in tickers)  # Прогнозируемая доходность портфеля
        vol = np.sqrt(sum((risks[ticker] ** 2) * (weights[ticker] ** 2) for ticker in tickers))  # Прогнозируемая волатильность портфеля
        sharpe = ret / vol if vol != 0 else 0  # Прогнозируемая метрика Шарпа

        # Передаем данные для обновления интерфейса
        window.write_event_value('-UPDATE_RESULTS-', (weight_str, ret, vol, sharpe, pie_chart_data))

        # Завершаем выполнение
        window.write_event_value('-LOG-UPDATE-', 'Расчеты завершены!')
    except Exception as e:
        window.write_event_value('-ERROR-', str(e))


# Функция для расчета доходности и риска (волатильности)
def calculate_returns_and_risks(tickers, start_date, end_date):
    # Загружаем данные о ценах для тикеров
    data = fetch_data(tickers, start_date, end_date)
    
    # Словарь для хранения результатов
    results = {}
    
    for ticker in tickers:
        if ticker in data and not data[ticker].empty:
            # Рассчитываем ежедневную доходность
            daily_returns = data[ticker].pct_change().dropna()

            # Рассчитываем среднюю годовую доходность
            annual_return = daily_returns.mean() * 252  # 252 торговых дня в году

            # Рассчитываем годовую волатильность
            annual_volatility = daily_returns.std() * np.sqrt(252)  # Умножаем на sqrt(252) для годовой волатильности
            
            # Добавляем результаты в словарь
            results[ticker] = {
                'return': round(annual_return * 100, 2),  # Переводим в проценты
                'risk': round(annual_volatility * 100, 2)  # Переводим в проценты
            }
        else:
            results[ticker] = {'return': None, 'risk': None}
    
    return results

# Основной цикл обработки событий
while True:
    event, values = window.read()

    if event == sg.WINDOW_CLOSED or event == 'Закрыть':
        break

    # --- Обработка событий первой вкладки ---
    if event == '-FILTER-':
        selected_sector = values['-SECTOR-']
        update_available_display(selected_sector)

    if event == '-AVAILABLE-' and values['-AVAILABLE-']:
        stock = values['-AVAILABLE-'][0].strip()
        for sector, stocks in current_sector_stocks.items():
            if stock in stocks:
                selected_stocks[sector].append(stock)
                stocks.remove(stock)
                undo_stack.append(('move', stock, sector))  # Добавляем в стек отмены
                break
        update_available_display(values['-SECTOR-'])
        update_selected_display()
        update_tickers_input()

    if event == '-SELECTED-' and values['-SELECTED-']:
        selected_stock = values['-SELECTED-'][0].strip()
        for sector, stocks in selected_stocks.items():
            if selected_stock in stocks:
                stocks.remove(selected_stock)
                current_sector_stocks[sector].append(selected_stock)
                undo_stack.append(('move_back', selected_stock, sector))  # Добавляем в стек отмены
                break
        update_available_display(values['-SECTOR-'])
        update_selected_display()
        update_tickers_input()

    if event == '-UNDO-' and undo_stack:
        last_action = undo_stack.pop()
        if last_action[0] == 'move':
            stock, sector = last_action[1], last_action[2]
            selected_stocks[sector].remove(stock)
            current_sector_stocks[sector].append(stock)
        elif last_action[0] == 'move_back':
            stock, sector = last_action[1], last_action[2]
            selected_stocks[sector].append(stock)
            current_sector_stocks[sector].remove(stock)
        update_available_display(values['-SECTOR-'])
        update_selected_display()
        update_tickers_input()

    if event == '-SELECT-ALL-':
        for sector, stocks in current_sector_stocks.items():
            selected_stocks[sector].extend(stocks)
            current_sector_stocks[sector] = []  # Очищаем доступные акции
        update_available_display(values['-SECTOR-'])
        update_selected_display()
        update_tickers_input()

    if event == '-SELECT-ALL-SECTOR-':
        selected_sector = values['-SECTOR-']
        if selected_sector == 'Все отрасли':
            sg.popup('Пожалуйста, выберите конкретную отрасль!')
        else:
            selected_stocks[selected_sector].extend(current_sector_stocks[selected_sector])
            current_sector_stocks[selected_sector] = []  # Очищаем доступные акции отрасли
            update_available_display(selected_sector)
            update_selected_display()
        update_tickers_input()

    if event == '-CLEAR-SELECTED-':
        for sector, stocks in selected_stocks.items():
            current_sector_stocks[sector].extend(stocks)
            stocks.clear()
        update_available_display(values['-SECTOR-'])
        update_selected_display()
        update_tickers_input()

    if event == '-SAVE-':
        save_selected_stocks(selected_stocks)

    # --- Обработка событий второй вкладки ---
    if event == 'Рассчитать оптимальные веса':
        # Очистка лога перед расчетом
        window['-LOG-'].update("Расчеты начались...\n")  # Добавляем сообщение о начале расчетов
        window.refresh()  # Принудительно обновляем интерфейс

        # Извлекаем тикеры
        tickers = [stock.strip() for sector, stocks in selected_stocks.items() for stock in stocks]

        if not tickers:
            sg.popup("Выберите хотя бы одну акцию для анализа!")
            continue

        # Проверка дат
        try:
            start_date = pd.to_datetime(values['start_date'])
            end_date = pd.to_datetime(values['end_date'])
            if start_date >= end_date:
                raise ValueError("Начальная дата должна быть меньше конечной!")
        except Exception as e:
            sg.popup(f"Ошибка ввода дат: {e}")
            continue

        try:
            # Расчет доходности и риска
            results = calculate_returns_and_risks(tickers, start_date, end_date)

            # Форматируем и выводим результаты доходности и риска
            risk_return_info = "\n".join(
                [f"{stock}: Доходность = {data['return']}%, Риск = {data['risk']}%"
                 for stock, data in results.items()]
            )
            window['-RISK-RETURN-'].update(risk_return_info)  # Выводим данные в окно "Доходность и риск"

            # Теперь добавим расчет ковариации и корреляции
            returns_data = [results[ticker]['return'] for ticker in tickers]  # Получаем доходности для всех тикеров
            covariance_matrix = np.cov(returns_data, rowvar=False)
            correlation_matrix = np.corrcoef(returns_data, rowvar=False)

            print("Covariance Matrix:")
            print(covariance_matrix)  # Выводим ковариационную матрицу
            print("Correlation Matrix:")
            print(correlation_matrix)  # Выводим корреляционную матрицу

            # Выводим акции в Multiline на вкладке 3
            window['-ANALYZED-STOCKS-'].update("\n".join(tickers))

            # Обновляем лог с сообщением "Расчеты завершены"
            window['-LOG-'].update("Расчеты завершены.\n", append=True)  # Добавляем сообщение о завершении расчетов
            window.refresh()  # Принудительно обновляем интерфейс

            # Делаем кнопку "Продолжить оптимизацию" активной
            window['-CONTINUE-OPTIMIZATION-'].update(disabled=False)
        except Exception as e:
            sg.popup_error("Ошибка при расчете доходности и риска!", str(e))
            window['-LOG-'].update(f"Ошибка: {e}\n", append=True)  # Добавляем ошибку в лог

    # --- Обработка событий третьей вкладки ---
    if event == '-CONTINUE-OPTIMIZATION-':
        # Создаем поток для оптимизации
        calculation_thread = threading.Thread(
            target=perform_optimization_in_thread,
            args=(window, tickers, start_date, end_date)
        )
        calculation_thread.start()
        window['-LOG-'].update("Оптимизация началась...\n")

    if event == '-UPDATE_RESULTS-':
        weight_str, ret, vol, sharpe, pie_chart_data = values[event]

        # Обновляем текстовые поля с результатами
        window['-RESULTS-'].update(
            f"Оптимальные веса портфеля:\n{weight_str}\n"
            f"Прогнозируемая месячная доходность: {ret:.2%}\n"
            f"Прогнозируемая волатильность: {vol:.2%}\n"
            f"Прогнозируемая метрика Шарпа: {sharpe:.2f}"
        )

        # Обновляем изображение для диаграммы
        if pie_chart_data:
            print(f"Updating pie chart with data of size: {len(pie_chart_data)} bytes")
            window['-PIE-'].update(data=pie_chart_data)  # Обновляем только данные изображения


        # Завершаем вывод в лог
        window['-LOG-'].print("Оптимизация завершена!")

    if event == '-ERROR-':
        sg.popup(f"Ошибка при оптимизации портфеля: {values[event]}")
        window['-LOG-'].print(f"Ошибка: {values[event]}")

    # Обработка нажатия на кнопку "Получить информацию о компании"
    if event == '-GET-INFO-':
        selected_ticker = values['-COMPANY-']
        if selected_ticker:
            company_info = companies_data.get(selected_ticker)
            if company_info:
                info_text = f"Компания: {company_info['company']}\n"
                info_text += f"Сектор: {company_info['sector']}\n"
                info_text += f"Отрасль: {company_info['industry']}\n"
                info_text += f"Описание: {company_info['description']}"
                window['-COMPANY-INFO-'].update(info_text)
            else:
                window['-COMPANY-INFO-'].update('Информация не найдена.')
        else:
            window['-COMPANY-INFO-'].update('Пожалуйста, выберите компанию.')

    if event == '-SEARCH-':
        entered_ticker = values['-STOCK-'].strip()
        if entered_ticker:
            company_info = companies_data.get(entered_ticker.upper())
            if company_info:
                info_text = f"Компания: {company_info['company']}\n"
                info_text += f"Сектор: {company_info['sector']}\n"
                info_text += f"Отрасль: {company_info['industry']}\n"
                info_text += f"Описание: {company_info['description']}"
                window['-COMPANY-INFO-'].update(info_text)
            else:
                window['-COMPANY-INFO-'].update('Тикер не найден. Убедитесь, что он введен корректно.')
        else:
            window['-COMPANY-INFO-'].update('Введите тикер для поиска.')


# Закрытие окна

window.close()


c:\Users\Кирилл\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


Заголовки столбцов: ['ticker', 'company', 'sector', 'industry', 'description']
